##### <span style="color:red">Name:</span>

Hiranmayi Duvvuri

951198138

# <span style="color:teal;">CIS 211 Project 8:  &nbsp; Weather Service</span>

##### Due 11:00 P.M. Friday June 5

##### Reading:  Perkovic Sec 11.1 -- 11.2

The project this week uses "REST" style web services to create a weather report. 

### `weather` 

Below you will find the defintion of a function named `weather` that prints a weather forecast for a specified location.

There are three different ways to call the function.  One is to pass a zip code:
<pre>
   weather(zip = 97405)
</pre>

A second way to print a forecast is to pass an address to the function:
<pre>
   weather(address = '1477 E 13th Ave, Eugene, OR')
</pre>

The third option is to pass latitude and longitude coordinates:
<pre>
   weather(lat = 44.0, long = -123.0)
</pre>

In each case the result will be the same, the current weather forecast for the specified location:
<pre>
   Showers likely and possibly a thunderstorm.  Cloudy, with a high near 63. Calm wind becoming 
   west southwest 5 to 7 mph in the afternoon.  Chance of precipitation is 70%. New rainfall 
   amounts between a tenth and quarter of an inch, except higher amounts possible in thunderstorms. 
</pre>

**Note:** The lecture notes for this week will explain how to write a function definition that allows users to call the function using keywords and values.

You have three jobs this week:
1. Write a helper function called `get_forecast` that will use a web service provided by NOAA to fetch the current weather forecast for a specified location.  
1. Since the NOAA service only works if you pass it a latitude and longitude, you need to write a second helper function, called `get_coords`, will look up a latitude and longitude given a zip code or address.
1. You need to test the top level `weather` function and explain how it handles errors.

### NOAA API 

A template for the URL for a web service provided by NOAA is
<pre>
http://forecast.weather.gov/MapClick.php?lat={}&lon={}&FcstType=json
</pre>
When numbers specifying latitude and longitude are substituted into this template the URL will return a JSON string with a complete forecst for that location.

For example, this shell command will fetch a weather report for Eugene:
<pre>
   % curl 'http://forecast.weather.gov/MapClick.php?lat=44.0&lon=-123.0&FcstType=json'
</pre>

When the result is translated as a JSON object you will get a Python dictionary.  One of the items in the dictionary is called `data`, which is itself a dictionary.  Inside that dictionary is an item called `text`, which is a list of strings.  The weather prediction you want is the first string in that list.

For example, if you save the result of the call to `json.loads` in a dictionary named `forecast`, this expression will get the string containing the prediction for the next few hours:
<pre>
forecast['data']['text'][0]
</pre>

### Geocoder API 

The NOAA service expects a location to be specified by its geographic coordinates (latitude and longitude).  If `weather` is called with a zip code or address we can use a service called a "geocoder" to look up the latitude and longitude.

There are two APIs for this service.  Use the first one if you have a zip code, use the second if you have an address:
<pre>
'http://geocoder.us/service/json?zip={}'
'http://geocoder.us/service/json?address={}&parse_address=1'
</pre>

For example, both of these shell commands return a JSON string containing the coordinates of the CS department:
<pre>
   % curl 'http://geocoder.us/service/json?zip=97403'
   % curl 'http://geocoder.us/service/json?address=1477+E.+13th+Ave.+Eugene,+OR&parse+address=1'
</pre>

When this JSON string is converted into a Python object you will get a list (some calls may specifiy ambiguous addresses or multiple zip codes).  You can assume the first item is the one you want.  This item will be a dictionary, and the coordinates are in the `lat` and `long` components of this dictionary.  

For example, if the result of the call to `json.loads` is saved in a list called `coords`, these expressions will look up the latitude and longitude of the specified location:
<pre>
coords[0]['lat']
coords[0]['long']
</pre>

### Top Level Function 

The following code cell contains the `import` statements for the Python libraries needed by this project and the definition of the top level `weather` function.

In [191]:
from urllib.request import urlopen
from urllib.error import HTTPError
from sys import argv
import json

def weather(**spec):
    try:
        if 'zipcode' in spec or 'address' in spec:
            lat, long = get_coords(**spec)
        elif not ('lat' in spec and 'long' in spec):
            raise Exception('specify zip, address, or latitude and longitude')
        else:
            lat, long = spec['lat'], spec['long']
        print(get_forecast(lat, long))
    except HTTPError:
        print('Incomplete or incorrect specification', spec)
    except Exception as err:
        print(err)

### Project 1:   `get_forecast` 

Write the function definition for the `get_forecast` helper function.  The funtion should take two integer arguments, the latitude and longitude, and return a single string, the weather forecast for that location.

##### <span style="color:red">Description:</span>

Formats the URL to put in the latitude and logitude given and urlopen accesses the webpage to get the weather and location data for the area. The data is fed to `json.loads` and converted to a usable dictionary in order to print only the forecast text from the data key.

##### <span style="color:red">Program:</span>

In [159]:
def get_forecast(lat, long):
    """
    gets the forecast given the latitude and logitude of an area.
    """
    wurl = 'http://forecast.weather.gov/MapClick.php?lat={}&lon={}&FcstType=json'.format(lat, long)
    weather = urlopen(wurl)
    wjson = weather.read().decode()
    forecast = json.loads(wjson)
    print(forecast['data']['text'][0])

##### <span style="color:red">Tests:</span>

In [195]:
get_forecast(44.0, -123.0)

Clear, with a low around 55. North northwest wind 5 to 9 mph becoming light northwest  after midnight. 


In [ ]:
"""
This cell shows the code during the debugging/testing process of the function, mostly using print statements to make sure
each part of the code works.
"""
    wurl = 'http://forecast.weather.gov/MapClick.php?lat={}&lon={}&FcstType=json'.format(lat, long)
    #print(wurl)
    weather = urlopen(wurl)
    #print(weather)
    wjson = weather.read().decode()
    #print(wjson)
    forecast = json.loads(wjson)
    #print(forecast)
    #print(type(forecast))
    print(forecast['data']['text'][0])

### Project 2:  `get_location` 

Write the function definition for the `get_location` helper function.  This function should allow users to either pass a zip code or an address, and the return value should be a pair of numbers corresponding to the latitude and longitude of the specified location.

**Note** This week's lecture notes will explain how to write a function that can be passed keyword arguments.  The top level `weather` function is an example -- this function can be passed arguments named `zip`, `address`, or `lat` and `lon`.


##### <span style="color:red">Description:</span>

Formats the URL based on if the keyword argument is the zipcode or the address for the location given and then urlopen accesses the webpage to get the location information. The data is fed to `json.loads` and converted to a usable dictionary in order to return the latitude and longitude of the area.

##### <span style="color:red">Program:</span>

In [224]:
def get_coords(**loc):
    """
    takes a zipcode or address and returns the latitude and longitude for that location.
    """
    if 'zipcode' in loc:
        url = 'http://geocoder.us/service/json?zip={}'.format(loc['zipcode'])
        
    elif 'address' in loc:
        addr = loc['address'].replace(' ', '+')
        url = 'http://geocoder.us/service/json?address={}&parse_address=1'.format(addr)
        
    content = urlopen(url).read().decode()
    coords = json.loads(content)
    return coords[0]['lat'], coords[0]['long']

##### <span style="color:red">Tests:</span>

In [222]:
get_coords(zipcode = 97405)

('44.004396', '-123.12203')

In [218]:
get_coords(address = '4001 SW Canyon Rd, Portland, OR')

('45.506940', '-122.718808')

In [ ]:
"""
Cell shows debugging/testing using print statements. 
"""
    if 'zipcode' in loc:
        url = 'http://geocoder.us/service/json?zip={}'.format(loc['zipcode'])
        print(loc)
        print(url)
        #urlopen(url)
        
    elif 'address' in loc:
        addr = loc['address'].replace(' ', '+')
        print(addr)
        url = 'http://geocoder.us/service/json?address={}&parse_address=1'.format(addr)
        print(url)
        
    content = urlopen(url).read().decode()
    coords = json.loads(content)
    print(coords[0]['lat'], coords[0]['long'])

### Project 3:  Test the `weather` Function 

In the space below use one or more code cells to make calls to `weather`.  Note that if you pass `lat` and `long` as arguments you can call `weather` even if you have not yet implemented the `get_location` helper.

Your tests should call `weather` with zip codes, addresses, and latitude/longitude.

After the code cells, write a brief description of how the function handles errors.  What happens if you pass an illegal or unknown zip code?  If you don't pass any valid arguments?

In [223]:
weather(zipcode = 97405)

Clear, with a low around 55. North northwest wind 6 to 11 mph. 
None


In [199]:
weather(zipcode = 97123)

Clear, with a low around 57. Northwest wind 5 to 10 mph. 
None


In [194]:
weather(address = '1477 E 13th Ave, Eugene, OR')

Clear, with a low around 57. North wind 5 to 11 mph. 
None


In [203]:
weather(address = '4001 SW Canyon Rd, Portland, OR')

Clear, with a low around 57. North northwest wind 6 to 11 mph. 
None


In [196]:
weather(lat = 44.0, long = -123.0)

Clear, with a low around 55. North northwest wind 5 to 9 mph becoming light northwest  after midnight. 
None


In [208]:
weather(lat = hey, long = go)

NameError: name 'hey' is not defined

In [211]:
weather(loc = 4)

specify zip, address, or latitude and longitude
